<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=81d36d825ad130ed58fa3e5561b6124834d49c2a4b422306572436e51e3e5204
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-12 14:28:13
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.50 L
Current:  1.45 C
-------------------
Today PnL: 1.10 L (0.76%)
Current PnL: -21.76 L (-14.26%)
CY Booked + Current PnL: -8.24 L (-5.4%)
-------------------
Total profit:  1.80 L
Total loss:  -23.56 L
-------------------
Total Booked + Current PnL: 18.74 L (14.92%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.33%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.33 C
Est FTT PnL: 88.43 L (61.04%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.3%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,101.49,51.0,M-SC,2.55,86833.0,-13944.0,14041.0,0.18,-13.84,16.17,0.09,245.0,-0.99,0.61,14.08,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-10.65,45.0,X-MC,3.30,88170.0,-2508.0,17414.0,-0.54,-2.77,19.75,16.44,101.0,-0.14,0.61,16.85,X40,ATH,PHARMA
34,ICICIGI,2252.93,-12.91,58.0,X-MC,7.57,188306.0,13986.0,21222.0,1.44,8.02,11.27,20.19,91.0,0.66,1.31,24.35,X40,ATH,INSURANCE
42,ITC,452.00,-38.34,46.0,X-LC,0.83,198876.0,-1262.0,22155.0,-0.04,-0.63,11.14,10.44,4.0,-0.06,1.39,4.90,X40,NTT,FMCG
38,INDIGOPNTS,1408.00,131.03,79.0,M-SC,4.40,151615.0,-22944.0,22970.0,1.68,-13.14,15.15,0.02,221.0,-1.00,1.06,31.88,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,ROUTE,2227.21,-56.81,40.0,H-SC,23.20,71094.0,-61972.0,153855.0,4.75,-46.57,216.41,69.05,118.0,-0.40,0.50,4.75,SR,ATH,IT
14,BSOFT,836.99,-9.37,69.0,H-SC,9.38,102013.0,-38640.0,111419.0,4.55,-27.47,109.22,51.74,131.0,-0.35,0.71,18.03,XR,ATH,IT
39,INDUSINDBK,1800.00,300.56,78.0,L-MC,7.11,49157.0,-28050.0,53443.0,4.39,-36.33,108.72,32.89,258.0,-0.52,0.34,35.37,XR,NTT,BANKS
69,SURYODAY,240.00,53.51,41.0,H-SC,13.75,140678.0,-38393.0,104116.0,3.36,-21.44,74.01,36.71,135.0,-0.37,0.98,39.58,XR,NTT,BANKS
79,VAIBHAVGBL,521.00,72.45,61.0,H-SC,6.65,149516.0,-33259.0,144328.0,2.93,-18.20,96.53,60.77,125.0,-0.23,1.04,34.65,XR,NTT,JEWELLERY


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VBL,671.64,-18.39,47.0,X-LC,5.68,293671.0,-22171.0,134824.0,-2.19,-7.02,45.91,35.67,5.0,-0.16,2.05,5.86,X40N,ATH,FMCG
70,SYMPHONY,1306.00,-45.32,40.0,M-SC,15.81,116649.0,-54492.0,54440.0,-1.26,-31.84,46.67,-0.03,196.0,-1.00,0.81,1.81,OX40N,NTT,DURABLES
75,TMPV,1065.00,NaN,50.0,NaN,NaN,210314.0,-187330.0,345609.0,-1.15,-47.11,164.33,39.81,54.0,-0.54,1.47,3.08,XY24,NTT,AUTO
11,BANDHANBNK,400.00,-19.45,34.0,H-SC,1.29,202260.0,-76299.0,333749.0,-1.02,-27.39,165.01,92.42,151.0,-0.23,1.41,16.89,XY24,NTT,BANKS
1,ABB,7934.00,-43.27,30.0,H-LC,5.05,236904.0,-24715.0,143919.0,-0.86,-9.45,60.75,45.57,7.0,-0.17,1.65,0.80,AR,NTT,ELECTRICAL


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-13.34,55.0,M-LC,5.72,152204.0,1259.0,108202.0,1.57,0.83,71.09,72.51,53.0,0.01,1.06,6.91,XR,NTT,IT
85,ZYDUSLIFE,1286.17,-9.05,39.0,H-MC,5.99,210105.0,531.0,74146.0,1.26,0.25,35.29,35.63,NaN,0.01,1.47,16.97,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIGOPNTS,1408.0,131.03,79.0,M-SC,4.40,151615.0,-22944.0,22970.0,1.68,-13.14,15.15,0.02,221.0,-1.00,1.06,31.88,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.0,101.49,51.0,M-SC,2.55,86833.0,-13944.0,14041.0,0.18,-13.84,16.17,0.09,245.0,-0.99,0.61,14.08,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-68.16,41.0,H-SC,2.30,219807.0,-49860.0,86186.0,0.16,-18.49,39.21,13.47,138.0,-0.58,1.53,11.77,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-24.21,50.0,H-MC,6.36,102619.0,-29216.0,71443.0,2.22,-22.16,69.62,32.03,98.0,-0.41,0.72,14.94,OX40N,NTT,IT
66,SIS,528.0,2033.16,49.0,H-SC,2.19,86088.0,-24944.0,48554.0,1.08,-22.47,56.40,21.26,156.0,-0.51,0.60,16.31,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,ZYDUSLIFE,1286.17,-9.05,39.0,H-MC,5.99,210105.0,531.0,74146.0,1.26,0.25,35.29,35.63,NaN,0.01,1.47,16.97,AR,ATH,PHARMA
84,WIPRO,420.00,-13.34,55.0,M-LC,5.72,152204.0,1259.0,108202.0,1.57,0.83,71.09,72.51,53.0,0.01,1.06,6.91,XR,NTT,IT
37,INDIAMART,4810.62,-52.40,61.0,H-SC,4.91,132765.0,9429.0,122197.0,2.16,7.65,92.04,106.72,119.0,0.08,0.93,32.81,AR,ATH,MISC
61,SAMMAANCAP,326.00,-192.05,54.0,M-SC,35.59,159147.0,9927.0,134256.0,0.27,6.65,84.36,96.62,208.0,0.07,1.11,72.23,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIAMART,4810.62,-52.40,61.0,H-SC,4.91,132765.0,9429.0,122197.0,2.16,7.65,92.04,106.72,119.0,0.08,0.93,32.81,AR,ATH,MISC
84,WIPRO,420.00,-13.34,55.0,M-LC,5.72,152204.0,1259.0,108202.0,1.57,0.83,71.09,72.51,53.0,0.01,1.06,6.91,XR,NTT,IT
85,ZYDUSLIFE,1286.17,-9.05,39.0,H-MC,5.99,210105.0,531.0,74146.0,1.26,0.25,35.29,35.63,NaN,0.01,1.47,16.97,AR,ATH,PHARMA
83,WHIRLPOOL,2270.00,-45.46,36.0,M-SC,8.83,112994.0,-1804.0,95842.0,-0.78,-1.57,84.82,81.92,223.0,-0.02,0.79,33.77,XR,NTT,DURABLES
47,KPIGREEN,731.05,4.03,47.0,H-SC,10.22,123757.0,-1540.0,60468.0,0.99,-1.23,48.86,47.03,141.0,-0.03,0.86,54.96,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.74,86926.0,-41744.0,87039.0,0.04,-32.44,100.13,35.20,219.0,-0.48,0.61,0.04,X40,NTT,FOOTWEAR
25,GILLETTE,11206.78,-17.30,27.0,X-MC,5.72,254070.0,174.0,82141.0,2.23,0.07,32.33,32.42,105.0,0.00,1.77,15.18,X40,ATH,FMCG
26,GLAXO,3466.20,-20.84,29.0,X-MC,4.86,200160.0,876.0,77142.0,1.04,0.44,38.54,39.15,96.0,0.01,1.40,29.57,X40,ATH,PHARMA
1,ABB,7934.00,-43.27,30.0,H-LC,5.05,236904.0,-24715.0,143919.0,-0.86,-9.45,60.75,45.57,7.0,-0.17,1.65,0.80,AR,NTT,ELECTRICAL
53,PGHH,17907.65,-32.93,31.0,X-MC,1.13,194235.0,-6585.0,74373.0,0.24,-3.28,38.29,33.76,80.0,-0.09,1.35,1.75,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,ITC,452.00,-38.34,46.0,X-LC,0.83,198876.0,-1262.0,22155.0,-0.04,-0.63,11.14,10.44,4.0,-0.06,1.39,4.90,X40,NTT,FMCG
53,PGHH,17907.65,-32.93,31.0,X-MC,1.13,194235.0,-6585.0,74373.0,0.24,-3.28,38.29,33.76,80.0,-0.09,1.35,1.75,X40,ATH,FMCG
65,SIEMENS,4671.50,-5.82,38.0,H-LC,1.68,151365.0,-34730.0,82206.0,0.18,-18.66,54.31,25.51,15.0,-0.42,1.06,11.54,AR,ATH,ELECTRICAL
30,HAVELLS,2069.16,-8.73,39.0,X-MC,3.31,304668.0,-23323.0,129850.0,0.50,-7.11,42.62,32.48,92.0,-0.18,2.13,3.50,X40,ATH,ELECTRICAL
78,UNITDSPR,1644.00,-5.95,63.0,X-LC,3.44,246424.0,12280.0,36348.0,1.29,5.24,14.75,20.77,86.0,0.34,1.72,12.53,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.74,86926.0,-41744.0,87039.0,0.04,-32.44,100.13,35.20,219.0,-0.48,0.61,0.04,X40,NTT,FOOTWEAR
54,QUESS,986.00,-51.52,32.0,X-SC,37.14,48079.0,-16927.0,167853.0,0.48,-26.04,349.12,232.18,198.0,-0.10,0.34,0.48,XY24,NTT,MISC
1,ABB,7934.00,-43.27,30.0,H-LC,5.05,236904.0,-24715.0,143919.0,-0.86,-9.45,60.75,45.57,7.0,-0.17,1.65,0.80,AR,NTT,ELECTRICAL
53,PGHH,17907.65,-32.93,31.0,X-MC,1.13,194235.0,-6585.0,74373.0,0.24,-3.28,38.29,33.76,80.0,-0.09,1.35,1.75,X40,ATH,FMCG
10,BAJAJHFL,181.50,-15.96,39.0,X-MC,6.32,177284.0,-23222.0,120376.0,0.39,-11.58,67.90,48.45,90.0,-0.19,1.24,2.23,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-51.52,32.0,X-SC,37.14,48079.0,-16927.0,167853.0,0.48,-26.04,349.12,232.18,198.0,-0.10,0.34,0.48,XY24,NTT,MISC
58,RELAXO,1176.00,-40.27,45.0,X-SC,3.70,81470.0,-63690.0,139615.0,0.34,-43.88,171.37,52.31,136.0,-0.46,0.57,9.05,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.65,45.0,X-MC,3.30,88170.0,-2508.0,17414.0,-0.54,-2.77,19.75,16.44,101.0,-0.14,0.61,16.85,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.74,86926.0,-41744.0,87039.0,0.04,-32.44,100.13,35.20,219.0,-0.48,0.61,0.04,X40,NTT,FOOTWEAR
33,HONAUT,58357.33,-24.48,36.0,X-SC,0.94,106500.0,-21438.0,68575.0,-0.01,-16.76,64.39,36.84,143.0,-0.31,0.74,9.03,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-26.44,63.0,X-LC,8.95,293656.0,-52300.0,118989.0,2.53,-15.12,40.52,19.28,1.0,-0.44,2.05,8.56,X40,ATH,IT
40,INFY,2275.00,-14.93,68.0,X-LC,4.54,330746.0,17858.0,153830.0,1.47,5.71,46.51,54.87,3.0,0.12,2.31,14.46,X40,BTT,IT
42,ITC,452.00,-38.34,46.0,X-LC,0.83,198876.0,-1262.0,22155.0,-0.04,-0.63,11.14,10.44,4.0,-0.06,1.39,4.90,X40,NTT,FMCG
81,VBL,671.64,-18.39,47.0,X-LC,5.68,293671.0,-22171.0,134824.0,-2.19,-7.02,45.91,35.67,5.0,-0.16,2.05,5.86,X40N,ATH,FMCG
1,ABB,7934.00,-43.27,30.0,H-LC,5.05,236904.0,-24715.0,143919.0,-0.86,-9.45,60.75,45.57,7.0,-0.17,1.65,0.80,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDUSINDBK,1800.00,300.56,78.0,L-MC,7.11,49157.0,-28050.0,53443.0,4.39,-36.33,108.72,32.89,258.0,-0.52,0.34,35.37,XR,NTT,BANKS
19,COFFEEDAY,80.00,-47.69,42.0,L-SC,26.24,76819.0,-36730.0,76781.0,0.28,-32.35,99.95,35.27,268.0,-0.48,0.54,88.02,XR,NTT,HOTELS
7,ASIANTILES,137.00,7627.78,69.0,L-SC,15.04,84779.0,-9031.0,85652.0,0.96,-9.63,101.03,81.67,269.0,-0.11,0.59,64.18,XR,NTT,CERAMICS
50,MASFIN,398.61,-18.70,51.0,H-SC,7.89,92400.0,-5580.0,27184.0,1.84,-5.70,29.42,22.05,152.0,-0.21,0.64,34.31,XR,ATH,FINANCE
47,KPIGREEN,731.05,4.03,47.0,H-SC,10.22,123757.0,-1540.0,60468.0,0.99,-1.23,48.86,47.03,141.0,-0.03,0.86,54.96,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,VAIBHAVGBL,521.00,72.45,61.0,H-SC,6.65,149516.0,-33259.0,144328.0,2.93,-18.20,96.53,60.77,125.0,-0.23,1.04,34.65,XR,NTT,JEWELLERY
7,ASIANTILES,137.00,7627.78,69.0,L-SC,15.04,84779.0,-9031.0,85652.0,0.96,-9.63,101.03,81.67,269.0,-0.11,0.59,64.18,XR,NTT,CERAMICS
38,INDIGOPNTS,1408.00,131.03,79.0,M-SC,4.40,151615.0,-22944.0,22970.0,1.68,-13.14,15.15,0.02,221.0,-1.00,1.06,31.88,OX40N,NTT,PAINTS
14,BSOFT,836.99,-9.37,69.0,H-SC,9.38,102013.0,-38640.0,111419.0,4.55,-27.47,109.22,51.74,131.0,-0.35,0.71,18.03,XR,ATH,IT
67,SONACOMS,806.63,-29.95,70.0,M-SC,6.17,87124.0,-14035.0,54034.0,1.64,-13.87,62.02,39.54,202.0,-0.26,0.61,22.85,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.91
1,25,44.51
2,50,76.23


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.56
LC    27.91
MC    27.07
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.45
X40      20.70
X40N     12.42
XY25     10.19
XR        9.75
AR        9.18
OX40N     6.58
X200      1.84
SR        1.04
MH        0.86
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.97
X-MC    22.91
X-LC    19.91
M-SC    12.53
X-SC     5.60
H-LC     4.55
H-MC     3.82
M-LC     2.45
L-SC     1.46
L-LC     1.00
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.85,-4.10,37.77
IT,12.81,-14.61,75.95
FINANCE,12.11,-8.77,58.07
MISC,7.70,-17.81,77.23
PAINTS,5.77,-11.57,28.19
ELECTRICAL,5.58,-13.03,53.11
INSURANCE,4.48,1.64,33.02
PHARMA,3.48,-0.22,33.85
AUTO,3.23,-50.66,113.03


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3229016.0,20
AR,1259149.0,10
XR,1257086.0,13
X40,1001680.0,14
X40N,688649.0,9
OX40N,541339.0,9
XY25,472942.0,7
SR,272586.0,2
MH,60468.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3393965.0,24
X-MC,1401886.0,16
M-SC,1248996.0,15
X-LC,797768.0,11
X-SC,547169.0,6
H-MC,325614.0,3
H-LC,285975.0,3
L-SC,251798.0,3
M-LC,154185.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1175553.0      6
           AR         836236.0      5
           XR         789721.0      7
M-SC       XY24       727681.0      6
X-MC       XY24       565392.0      4
           X40        493827.0      8
X-LC       X40        420814.0      5
X-SC       X40N       292277.0      4
H-SC       SR         272586.0      2
           OX40N      259401.0      3
X-LC       X40N       237185.0      3
H-LC       AR         226125.0      2
X-MC       XY25       183480.0      2
H-MC       XY24       180025.0      1
X-SC       XY24       167853.0      1
L-SC       XR         162433.0      2
X-MC       X40N       159187.0      2
M-SC       XR         143287.0      2
           XY25       134256.0      1
           AR         122642.0      2
           OX40N      121130.0      4
M-LC       XR         108202.0      1
L-SC       OX40N       89365.0      1
X-SC       X40         87039.0      1
H-MC       AR          74146.0      1
X-LC       XY25        72866.0      2
H-MC       OX40N       71443.0      1
X-LC       XY24        66903.0      1
H-SC       MH          60468.0      1
H-LC       X200        59850.0      1
L-MC       XR          53443.0      1
M-LC       XY25        45983.0      1
L-LC       XY25        36357.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 58.0 seconds
